In [1]:
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms import Tongyi

llm = Tongyi(temperature=0.7)

template = """我希望你能充当新公司的命名顾问。一个生产{product}的公司好的中文名字是什么,请给出五个选项？"""
prompt = ChatPromptTemplate.from_template(template)
chain = (
        {"product": RunnablePassthrough()}
        | prompt
        | llm
)
chain.invoke("彩色袜子")

'当然可以，以下是我为您的彩色袜子公司提出的五个中文名字建议：\n\n1. 彩韵袜业：这个名字融合了“色彩”和“韵律”的概念，暗示了袜子的多彩和时尚感。\n2. 飞彩足下：寓意袜子如彩虹般绚丽，为双脚增添活力与乐趣。\n3. 明彩袜坊：明快的色彩，手工的艺术，表达出产品的质量和个性。\n4. 舒彩履梦：强调舒适度的同时，也表达了穿上这些袜子如同实现梦想般的美好。\n5. 彩织生活：体现出袜子是日常生活的一部分，且充满色彩，富有生活气息。\n\n希望这些建议能对您有所帮助！如果您需要更多的选择或者有特定的要求，欢迎继续告诉我。'

In [4]:
from langchain_community.llms import Tongyi

llm = Tongyi(temperature=0.7)

In [6]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain_community.chat_models.tongyi import ChatTongyi

model = ChatTongyi()

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

prompt2 = PromptTemplate(
    template="请列出五个 {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)
chain2 = prompt2 | llm | output_parser

chain2.invoke("中国的名山")

['泰山', '黄山', '衡山', '华山', '嵩山']

In [10]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.vectorstores import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.chat_models.tongyi import ChatTongyi

embedding = SentenceTransformerEmbeddings(model_name='D:\workspace\project\LLM\models\m3e-base')

vectorstore = Chroma.from_texts(
    ["小明在华为工作"], embedding=embedding
)
retriever = vectorstore.as_retriever()
template = """仅根据以下上下文回答问题：
{context}

问题：{question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatTongyi()

retrieval_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser()
)

In [11]:
retrieval_chain.invoke("小强在哪里工作？")

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


'根据提供的上下文，无法回答小强在哪里工作的问题，因为只有关于小明在华为工作的信息。'

In [8]:
from langchain_community.llms import Tongyi

llm = Tongyi(temperature=0.7)

from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""使用下面的语料来回答本模板最末尾的问题。如果你不知道问题的答案，直接回答 "我不知道"，禁止随意编造答案。
        为了保证答案尽可能简洁，你的回答必须不超过三句话，你的回答中不可以带有星号。
        请注意！在每次回答结束之后，你都必须接上 "感谢你的提问" 作为结束语
        以下是一对问题和答案的样例：
            请问：秦始皇的原名是什么
            秦始皇原名嬴政。感谢你的提问。
        
        以下是语料：
<context>
{context}
</context>

Question: {input}""")

from langchain.document_loaders import TextLoader

loader_txt = TextLoader(r'D:\workspace\project\learn-project\python_data_course\大模型产品开发导论\云岚宗.txt',
                        encoding='utf8')
docs_txt = loader_txt.load()
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=384, chunk_overlap=0,
                                               separators=["\n\n", "\n", " ", "", "。", "，"])
docs_prepared = text_splitter.split_documents(docs_txt)
from langchain.embeddings import SentenceTransformerEmbeddings

embedding = SentenceTransformerEmbeddings(model_name='D:\workspace\project\LLM\models\m3e-base')
from langchain.vectorstores import Chroma

vectordb = Chroma.from_documents(documents=docs_prepared, embedding=embedding)
retriever = vectordb.as_retriever()
#创建检索链
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, document_chain)



D:\Program\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
D:\Program\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [9]:
response = retrieval_chain.invoke({
    "input": "纳兰桀是谁?"
})
print(response["answer"])

纳兰桀是加玛帝国狮心元帅，纳兰嫣然的祖父。感谢你的提问。


In [13]:
from langchain_core.runnables import RunnablePassthrough

chain2 = ({"input": RunnablePassthrough(), "context": retriever} | prompt | llm)
chain2.invoke("纳兰桀是谁?")

'纳兰桀是纳兰嫣然的祖父，他在家族中有绝对的话语权。感谢你的提问。'

In [13]:
from langchain.document_loaders import Doc